In [1]:
import pandas as pd

In [2]:
data = pd.read_csv("NIFTY 50.csv")
data.head()

,date,open,high,low,close,volume
0,2015-01-09 09:15:00,8285.45,8295.90,8285.45,8292.10,0
1,2015-01-09 09:16:00,8292.60,8293.60,8287.20,8288.15,0
2,2015-01-09 09:17:00,8287.40,8293.90,8287.40,8293.90,0
3,2015-01-09 09:18:00,8294.25,8300.65,8293.90,8300.65,0
4,2015-01-09 09:19:00,8300.60,8301.30,8298.75,8301.20,0


In [3]:
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')

data['open'] = data['open'].resample('3Min').first()
data['high'] = data['high'].resample('3Min').max()
data['low'] = data['low'].resample('3Min').min()
data['close'] = data['close'].resample('3Min').last()
data = data.dropna()

data = data.drop('volume', axis=1)
data['bar_marker'] = pd.Series()
data['sph_spl_marker'] = pd.Series()
data['place_order_marker'] = pd.Series()

data.head()

,open,high,low,close,bar_marker,sph_spl_marker,place_order_marker
date,,,,,,,
2015-01-09 09:15:00,8285.45,8295.90,8285.45,8293.90,NaN,NaN,NaN
2015-01-09 09:18:00,8294.25,8303.00,8293.90,8300.00,NaN,NaN,NaN
2015-01-09 09:21:00,8300.65,8302.90,8293.25,8294.70,NaN,NaN,NaN
2015-01-09 09:24:00,8295.20,8302.55,8289.45,8289.45,NaN,NaN,NaN
2015-01-09 09:27:00,8289.65,8294.15,8286.80,8294.15,NaN,NaN,NaN


In [4]:
data.loc[data.index[0],'bar_marker'] = 'AB'
B1L_marked = ''
B2L_marked = ''
B1H_marked = ''
B2H_marked = ''
place_order_lst = []

In [5]:
for i in range(1, 300):
    if data.loc[data['bar_marker'] == 'AB', 'low'].item() > data.loc[data.index[i], 'low'] and data.loc[data['bar_marker'] == 'AB', 'close'].item() > data.loc[data.index[i], 'close']:
        if B1L_marked != 'Yes':
            data.loc[data.index[i], 'bar_marker'] = 'B1L'
            B1L_marked = 'Yes'
        else:
            data.loc[data.index[i], 'bar_marker'] = 'B2L'
            B2L_marked = 'Yes'
    elif data.loc[data['bar_marker'] == 'AB', 'high'].item() < data.loc[data.index[i], 'high'] and data.loc[data['bar_marker'] == 'AB', 'close'].item() < data.loc[data.index[i], 'close']:
        if B1H_marked != 'Yes':
            data.loc[data.index[i], 'bar_marker'] = 'B1H'
            B1H_marked = 'Yes'
        else:
            data.loc[data.index[i], 'bar_marker'] = 'B2H'
            B2H_marked = 'Yes'

    if B2L_marked == 'Yes':
        data.loc[data['bar_marker'] == 'AB', 'sph_spl_marker'] = 'H'
        data.loc[data['bar_marker'] == 'B1L', 'bar_marker'] = 'B1L_old'
        B1L_marked = ''
        B2L_marked = ''
        place_order_lst.append('H')
        if len(place_order_lst) == 3:
            if place_order_lst == ['L','H','L'] or place_order_lst == ['H','L','H']:
                data.loc[data['bar_marker'] == 'AB', 'place_order_marker'] = 'Yes'
            del place_order_lst[0]
        data.loc[data['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
        data.loc[data['bar_marker'] == 'B2L', 'bar_marker'] = 'AB'

    if B2H_marked == 'Yes':
        data.loc[data['bar_marker'] == 'AB', 'sph_spl_marker'] = 'L'
        data.loc[data['bar_marker'] == 'B1H', 'bar_marker'] = 'B1H_old'
        B1H_marked = ''
        B2H_marked = ''
        place_order_lst.append('L')
        if len(place_order_lst) == 3:
            if place_order_lst == ['L','H','L'] or place_order_lst == ['H','L','H']:
                data.loc[data['bar_marker'] == 'AB', 'place_order_marker'] = 'Yes'
            del place_order_lst[0]
        data.loc[data['bar_marker'] == 'AB', 'bar_marker'] = 'AB_old'
        data.loc[data['bar_marker'] == 'B2H', 'bar_marker'] = 'AB'

In [6]:
#print('B1L_marked = ' + B1L_marked + ' B2L_marked = ' + B2L_marked + ' B1H_marked = ' + B1H_marked + ' B2H_marked = ' + B2H_marked + ' place_order_lst = ' + str(place_order_lst))

In [7]:
data.head(300)

,open,high,low,close,bar_marker,sph_spl_marker,place_order_marker
date,,,,,,,
2015-01-09 09:15:00,8285.45,8295.90,8285.45,8293.90,AB_old,L,NaN
2015-01-09 09:18:00,8294.25,8303.00,8293.90,8300.00,B1H_old,NaN,NaN
2015-01-09 09:21:00,8300.65,8302.90,8293.25,8294.70,AB_old,H,NaN
2015-01-09 09:24:00,8295.20,8302.55,8289.45,8289.45,B1L_old,NaN,NaN
2015-01-09 09:27:00,8289.65,8294.15,8286.80,8294.15,AB_old,H,NaN
2015-01-09 09:30:00,8294.10,8295.75,8282.00,8283.10,B1L_old,NaN,NaN
2015-01-09 09:33:00,8282.75,8290.45,8280.65,8285.25,AB_old,H,NaN
2015-01-09 09:36:00,8284.75,8287.55,8278.00,8282.55,B1L_old,NaN,NaN
2015-01-09 09:39:00,8282.30,8288.30,8281.20,8285.60,NaN,NaN,NaN
